In [1]:
import logging

import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

In [2]:
logging.getLogger("tensorflow").setLevel(logging.ERROR)

https://github.com/tensorflow/tensorflow/issues/29781#issuecomment-504980912

In [3]:
def f(v):
    return tf.sin(v[0] + tf.cos(v[1]))

In [4]:
tf.random.set_seed(0)
x = tf.Variable(tf.random.normal((2,)))

with tf.GradientTape(persistent=True) as tape:
    y = f(x)
    grad = tape.gradient(y, x)
hessian = tape.jacobian(grad, x)

print(x)
print(grad)
print(hessian)

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1.5110626 , 0.42292204], dtype=float32)>
tf.Tensor([-0.75270414  0.30892995], shape=(2,), dtype=float32)
tf.Tensor(
[[-0.65835893  0.27020815]
 [ 0.27020815  0.5754854 ]], shape=(2, 2), dtype=float32)


In [5]:
def hessian(f, x):
    with tf.GradientTape(persistent=True) as tape:
        y = f(x)
        grad = tape.gradient(y, x)
    hessian = tape.jacobian(grad, x)
    return hessian

tf.random.set_seed(0)
x = tf.Variable(tf.random.normal((2,)))

print(x)
print(f(x))
print(hessian(f, x))

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1.5110626 , 0.42292204], dtype=float32)>
tf.Tensor(0.65835893, shape=(), dtype=float32)
tf.Tensor(
[[-0.65835893  0.27020815]
 [ 0.27020815  0.5754854 ]], shape=(2, 2), dtype=float32)


In [6]:
def hessian2(f, x):
    with tf.GradientTape(persistent=True) as tape:
        y = f(x)
        grad = tape.gradient(y, x)
    hessian = tape.jacobian(grad, x)
    return hessian

tf.random.set_seed(0)
x = tf.Variable(tf.random.normal((2,)))

print(hessian2(f, x))

tf.Tensor(
[[-0.65835893  0.27020815]
 [ 0.27020815  0.5754854 ]], shape=(2, 2), dtype=float32)


# also works, but slower

In [7]:
tf.random.set_seed(0)
x = tf.Variable(tf.random.normal((2,)))

with tf.GradientTape(persistent=True) as hess_tape:
    with tf.GradientTape() as grad_tape:
        y = f(x)
    grad = grad_tape.gradient(y, x)
    grad_grads = (hess_tape.gradient(g, x) for g in grad)
    hess_columns = [gg[tf.newaxis, ...] for gg in grad_grads]
    hessian = tf.concat(hess_columns, axis=0)

print(x)
print(grad)
print(hessian)

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1.5110626 , 0.42292204], dtype=float32)>
tf.Tensor([-0.75270414  0.30892995], shape=(2,), dtype=float32)
tf.Tensor(
[[-0.65835893  0.27020815]
 [ 0.27020815  0.5754854 ]], shape=(2, 2), dtype=float32)


In [8]:
def hessian3(f, x):
    with tf.GradientTape(persistent=True) as hess_tape:
        with tf.GradientTape() as grad_tape:
            y = f(x)
        grad = grad_tape.gradient(y, x)
        grad_grads = [hess_tape.gradient(g, x) for g in grad]
    hess_rows = [gg[tf.newaxis, ...] for gg in grad_grads]
    hessian = tf.concat(hess_rows, axis=0)
    return hessian

tf.random.set_seed(0)
x = tf.Variable(tf.random.normal((2,)))

print(hessian3(f, x))

tf.Tensor(
[[-0.65835893  0.27020815]
 [ 0.27020815  0.5754854 ]], shape=(2, 2), dtype=float32)


# Didn't work

In [9]:
tf.random.set_seed(0)
v = tf.Variable(tf.random.normal((2,)))

with tf.GradientTape(persistent=True) as tape:
    y = tf.sin(v[0] + tf.cos(v[1]))
    grads = tape.gradient(y, [v])

hessians = tape.gradient(grads[0], [v])
print(hessians)

[<tf.Tensor: id=1542, shape=(2,), dtype=float32, numpy=array([-0.38815078,  0.8456935 ], dtype=float32)>]


In [10]:
tf.random.set_seed(0)
v = tf.Variable(tf.random.normal((2,)))

with tf.GradientTape(persistent=True) as tape:
    y = tf.sin(v[0] + tf.cos(v[1]))
    grad = tape.gradient(y, v)
    hessian = tape.gradient(grad, v)
    hessian0 = tape.gradient(grad[0], v)
    hessian1 = tape.gradient(grad[1], v)

print(hessian)
print(hessian0)
print(hessian1)
print(hessian0 + hessian1)

tf.Tensor([-0.38815078  0.8456935 ], shape=(2,), dtype=float32)
tf.Tensor([-0.65835893  0.27020815], shape=(2,), dtype=float32)
tf.Tensor([0.27020815 0.5754854 ], shape=(2,), dtype=float32)
tf.Tensor([-0.38815078  0.8456936 ], shape=(2,), dtype=float32)


In [11]:
tf.random.set_seed(0)
v = tf.Variable(tf.random.normal((2,)))

with tf.GradientTape(persistent=True) as tape2:
    with tf.GradientTape(persistent=True) as tape:
        y = tf.sin(v[0] + tf.cos(v[1]))
    jacobian = tape.jacobian(y, v)
    print(jacobian)
    print(tape2.gradient(jacobian, v))
    j_t = tf.transpose(jacobian)
    print(tape2.gradient(j_t, v))
    j_unstacked = tf.unstack(jacobian)
    print(j_unstacked)
    print(tape2.gradient(j_unstacked, v))


tf.Tensor([-0.75270414  0.30892995], shape=(2,), dtype=float32)
tf.Tensor([-0.38815078  0.8456935 ], shape=(2,), dtype=float32)
tf.Tensor([-0.38815078  0.8456935 ], shape=(2,), dtype=float32)
[<tf.Tensor: id=2411, shape=(), dtype=float32, numpy=-0.75270414>, <tf.Tensor: id=2412, shape=(), dtype=float32, numpy=0.30892995>]
tf.Tensor([-0.38815078  0.8456935 ], shape=(2,), dtype=float32)


In [12]:
tf.random.set_seed(0)
v = tf.Variable(tf.random.normal((2,)))
with tf.GradientTape(persistent=True) as tape:
    y = tf.sin(v[0] + tf.cos(v[1]))
    grad = tape.gradient(y, v)

hessian = tape.gradient(grad, v)

print("grad")
print(grad)
print("hessian")
print(hessian)

grad
tf.Tensor([-0.75270414  0.30892995], shape=(2,), dtype=float32)
hessian
tf.Tensor([-0.38815078  0.8456935 ], shape=(2,), dtype=float32)


In [13]:
tf.random.set_seed(0)
v = tf.Variable(tf.random.normal((2,)))

with tf.GradientTape(persistent=True) as tape:
    y = tf.sin(v[0] + tf.cos(v[1]))
    grad = tape.gradient(y, v)
    hessian = tape.gradient(grad, v)
    hessian0 = tape.gradient(grad[0], v)
    hessian1 = tape.gradient(grad[1], v)
    
print("hessian:", hessian)
print("hessian0:", hessian0)
print("hessian1:", hessian1)
print("hessian0 + hessian1:", hessian0 + hessian1)

hessian: tf.Tensor([-0.38815078  0.8456935 ], shape=(2,), dtype=float32)
hessian0: tf.Tensor([-0.65835893  0.27020815], shape=(2,), dtype=float32)
hessian1: tf.Tensor([0.27020815 0.5754854 ], shape=(2,), dtype=float32)
hessian0 + hessian1: tf.Tensor([-0.38815078  0.8456936 ], shape=(2,), dtype=float32)


In [14]:
tf.random.set_seed(0)
v = tf.Variable(tf.random.normal((2,)))

with tf.GradientTape(persistent=True) as tape:
    y = tf.sin(v[0] + tf.cos(v[1]))
    grads = tape.gradient(y, [v])
    print("grads", grads)
    hess0 = tape.gradient(grads[0][0], [v])
    print("hess0", hess0)
    hess1 = tape.gradient(grads[0][1], [v])
    print("hess1", hess1)

hessians = tape.gradient(grads[0], [v])
print("hessians", hessians)

grads [<tf.Tensor: id=2662, shape=(2,), dtype=float32, numpy=array([-0.75270414,  0.30892995], dtype=float32)>]
hess0 [<tf.Tensor: id=2691, shape=(2,), dtype=float32, numpy=array([-0.65835893,  0.27020815], dtype=float32)>]
hess1 [<tf.Tensor: id=2720, shape=(2,), dtype=float32, numpy=array([0.27020815, 0.5754854 ], dtype=float32)>]
hessians [<tf.Tensor: id=2745, shape=(2,), dtype=float32, numpy=array([-0.38815078,  0.8456935 ], dtype=float32)>]


In [15]:
def g(x):
    return (x[0] + 3 * x[1]) ** 2

In [16]:
tf.random.set_seed(0)
v = tf.Variable(tf.constant([1, 2], dtype=tf.dtypes.float32))

with tf.GradientTape(persistent=True) as tape:
    y = g(v)
    jac = tape.jacobian(y, v)
    print("jac", jac)

print(tape.gradient(jac, v))


ValueError: Tape is already recording.

### did work, eventually

In [18]:
tf.random.set_seed(0)
v = tf.Variable(tf.constant([1, 2], dtype=tf.dtypes.float32))

with tf.GradientTape(persistent=True) as tape:
    y = g(v)
    grad = tape.gradient(y, v)
    print("grad", grad)

jacobian = tape.jacobian(y, v)
print(tape.gradient(jacobian, v))
print(tape.gradient(grad, v))
print(tape.jacobian(grad, v))


grad tf.Tensor([14. 42.], shape=(2,), dtype=float32)
None
tf.Tensor([ 8. 24.], shape=(2,), dtype=float32)
tf.Tensor(
[[ 2.  6.]
 [ 6. 18.]], shape=(2, 2), dtype=float32)
